In [19]:
# Prepare Library
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk

# REF
# https://www.youtube.com/watch?v=W3DzSm8WI1g


In [20]:
# DEFINE CONSTRAINT
WIDHT = 800
HEIGHT = 600

In [21]:
def scan_detection(image, lower_threshold, upper_threshold):
    document_contour = np.array([[0, 0], [WIDHT, 0], [WIDHT, HEIGHT], [0, HEIGHT]])

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, threshold = cv2.threshold(blur, lower_threshold, upper_threshold, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    cv2.imshow("gray(blur)", blur)
    cv2.imshow("threshold", threshold)

    contours, _ = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    max_area = 0
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 1000:
            peri = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.015 * peri, True)

            if area > max_area and len(approx) == 4:
                document_contour = approx
                max_area = area

    cv2.drawContours(frame, [document_contour], -1, (0, 255, 0), 3)


V1 - TANPA SETTINGAN THRESHOLD

In [26]:
# SETUP VIDEO (OPEN-CV)
cap = cv2.VideoCapture(1 )
cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDHT)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)

while True:
    _, frame = cap.read()
    # -- Kalau mau di rotate -- 
    # frame = cv2.rotate(frame, cv2.ROTATE_180)
    frame_copy = frame.copy()

    lower_thresh = 0
    upper_thresh = 255
    scan_detection(frame_copy, lower_thresh, upper_thresh)


    cv2.imshow("input", frame)

    # ESC button
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

V2 - DENGAN SETTINGAN THRESHOLD

In [24]:
# SETUP VIDEO (OpenCV)
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDHT)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)

# Create Tkinter GUI
root = tk.Tk()
root.title("Threshold Settings")
root.geometry("400x100")

# Create a scale for lower threshold
lower_thresh_label = ttk.Label(root, text="Lower Threshold")
lower_thresh_label.pack()

lower_thresh_scale = ttk.Scale(root, from_=0, to=255, length=300, orient="horizontal")
lower_thresh_scale.set(50)  # Initial value
lower_thresh_scale.pack()

# Create a scale for upper threshold
upper_thresh_label = ttk.Label(root, text="Upper Threshold")
upper_thresh_label.pack()

upper_thresh_scale = ttk.Scale(root, from_=0, to=255, length=300, orient="horizontal")
upper_thresh_scale.set(200)  # Initial value
upper_thresh_scale.pack()

def update_threshold():
    _, frame = cap.read()
    frame_copy = frame.copy()

    lower_thresh = lower_thresh_scale.get()
    upper_thresh = upper_thresh_scale.get()

    scan_detection(frame_copy, lower_thresh, upper_thresh)
    cv2.imshow("input", frame)

    # Call this function again after 1ms
    root.after(1, update_threshold)

# Call the update_threshold function initially
update_threshold()

# Start Tkinter event loop
root.mainloop()

# Release video capture and destroy windows
cap.release()
cv2.destroyAllWindows()